---
# Name of Method - Model 2
---
In this notebook, ...


## Results

---

### Importing necessary library

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Read data file

In [2]:
churn_df = pd.read_excel('../data/churn_cleaned.xlsx')
churn_df

,Unnamed: 0,City,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,...,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Churn Value,Churn Reason
0,0,Los Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,1,Competitors
1,1,Los Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,1,Moved
2,2,Los Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,1,Moved
3,3,Los Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,1,Moved
4,4,Los Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,...,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,1,Competitors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,7038,Landers,92285,34.341737,-116.539416,Female,No,No,No,72,...,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),21.15,0,Not applicable
7039,7039,Adelanto,92301,34.667815,-117.536183,Male,No,Yes,Yes,24,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,0,Not applicable
7040,7040,Amboy,92304,34.559882,-115.637164,Female,No,Yes,Yes,72,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,0,Not applicable
7041,7041,Angelus Oaks,92305,34.167800,-116.864330,Female,No,Yes,Yes,11,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,0,Not applicable


---

<center>
    
## Preparing data

</center>

---

### Separate X and y features

In [4]:
# Seperate X and y features
X = churn_df.drop(columns=['Churn Value'])
y = churn_df['Churn Value']

### Split dataset (training/testing)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=5)

### Converting data subset to dataframe 

In [6]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

### Scale X features

In [ ]:
# Create instance of scaler
scaler = StandardScaler()

# Scale the data
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

---

<center>
    
## Model

</center>

---

### Find best parameters for model


### Run model on training dataset

### Investigate best model's predictive features

---

<center>
    
## Validation

</center>

---

### Run model on testing dataset

### Model's validation

---

<center>
    
## Results

</center>

---

### Results